In [2]:
import ee
import datetime


In [9]:
# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-sfranze1417')


# start_date = '2021-01-01'
# end_date   = '2024-04-01'

start_date = '2020-12-31'
end_date   = '2025-12-31'

label = 'b5'

In [10]:
# Load your ROI (replace with your actual asset path)
roi = ee.FeatureCollection("projects/ee-sfranze1417/assets/TZ-Usangu/s2_"+label)

# Optional overlays
wetland_fans = ee.FeatureCollection(
    "projects/ee-sfranze1417/assets/TZ-Usangu/wetland_fans_domain_37S"
)

rivers = ee.FeatureCollection(
    "projects/ee-sfranze1417/assets/TZ-Usangu/river_network_manual_dissolved_smoothed"
)

In [11]:
## Functions
def mask_s2_clouds(image):
    qa = image.select('QA60')

    cloud_bit_mask  = 1 << 10
    cirrus_bit_mask = 1 << 11

    mask = (
        qa.bitwiseAnd(cloud_bit_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    return (
        image.updateMask(mask)
        .divide(10000)
        .copyProperties(image, ['system:time_start'])
    )


In [12]:
## Load S2
s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(start_date, end_date)
    .filterBounds(roi)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

n_images = s2.size().getInfo()
print(f'Found {n_images} Sentinel-2 images')


Found 380 Sentinel-2 images


In [13]:
## Visualization parameters
rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 0.3
}

## Rasterized outlines
wetland_outline = ee.Image().paint(
    featureCollection=wetland_fans,
    color=1,
    width=2
).visualize(
    palette=['FFFF00'],  # yellow
    forceRgbOutput=True
)

river_outline = ee.Image().paint(
    featureCollection=rivers,
    color=1,
    width=1
).visualize(
    palette=['0000FF'],  # blue
    forceRgbOutput=True
)



In [14]:
## Loop over images and export
image_list = s2.toList(n_images)

for i in range(n_images):
    image = ee.Image(image_list.get(i))

    # Format date
    date_str = (
        ee.Date(image.get('system:time_start'))
        .format('YYYY-MM-dd')
        .getInfo()
    )

    print(f'Processing Sentinel-2 image: {date_str}')

    # Visualize RGB
    rgb = image.visualize(**rgb_vis).clip(roi)

    # Blend outlines
    final_vis = (
        rgb
        .blend(wetland_outline)
        .blend(river_outline)
    )

    # Export
    task = ee.batch.Export.image.toDrive(
        image=final_vis,
        description=f'S2_RGB_{date_str}_{label}',
        folder=f'GEE_S2_Visuals/{date_str[:4]}/{label}',
        fileNamePrefix=f'S2_RGB_{date_str}_{label}',
        region=roi.geometry(),
        scale=10,
        crs='EPSG:4326',
        maxPixels=1e13
    )

    task.start()
    print(f'  → Started export task for {date_str}')



Processing Sentinel-2 image: 2021-01-18
  → Started export task for 2021-01-18
Processing Sentinel-2 image: 2021-01-28
  → Started export task for 2021-01-28
Processing Sentinel-2 image: 2021-02-05
  → Started export task for 2021-02-05
Processing Sentinel-2 image: 2021-02-15
  → Started export task for 2021-02-15
Processing Sentinel-2 image: 2021-02-20
  → Started export task for 2021-02-20
Processing Sentinel-2 image: 2021-03-02
  → Started export task for 2021-03-02
Processing Sentinel-2 image: 2021-03-12
  → Started export task for 2021-03-12
Processing Sentinel-2 image: 2021-03-14
  → Started export task for 2021-03-14
Processing Sentinel-2 image: 2021-03-24
  → Started export task for 2021-03-24
Processing Sentinel-2 image: 2021-04-08
  → Started export task for 2021-04-08
Processing Sentinel-2 image: 2021-04-23
  → Started export task for 2021-04-23
Processing Sentinel-2 image: 2021-05-06
  → Started export task for 2021-05-06
Processing Sentinel-2 image: 2021-05-08
  → Started 